# Modul 8 Persamaan Diferensial Numerik: PDP Parabolik (persamaan panas/difusi)

Kembali ke [Persamaan Diferensial Numerik](./pdnum2024genap.qmd)

Bentuk umum PDP parabolik (lebih tepatnya persamaan panas atau persamaan difusi) bisa ditulis sebagai berikut:

$$\frac{\partial u}{\partial t} (x,t) = \alpha^2 \frac{\partial^2 u}{\partial x^2}(x,t), \quad 0 < x < l, \quad 0 < t < T,$$

dengan syarat batas serta nilai awal

$$u(0,t) = u(l,t) = 0, \quad 0 < t < T, \quad \text{dan} \quad u(x,0) = f(x), \quad 0 \le x \le l$$

*Step size* dalam variabel $x$ bisa ditulis $h = \Delta x = l/m$ untuk suatu bilangan bulat positif $m$.

*Step size* dalam variabel $t$ bisa ditulis $k = \Delta t = T/N$ untuk suatu bilangan bulat positif $N$.

## Metode *Forward Difference* / Metode Eksplisit untuk PDP Parabolik

### Ide utama

Langkah-langkah metode *forward difference* bisa ditulis sebagai berikut.

1. Misalkan

$$\lambda = \alpha^2 \left(\frac{k}{h^2}\right)$$

2. Susun matriks tridiagonal $A$ berukuran $(m-1) \times (m-1)$ sebagai berikut:

$$A = \begin{bmatrix}
    (1-2\lambda) & \lambda & 0 & \cdots & \cdots & \cdots & \cdots & 0 \\
    \lambda & (1-2\lambda) & \lambda & 0 & \ddots & \ddots & \ddots & \vdots \\
    0 & \lambda & (1-2\lambda) & \lambda & 0 & \ddots & \ddots & \vdots \\
    \vdots & \ddots & \ddots & \ddots & \ddots & \ddots & \ddots & \vdots \\
    \vdots & \ddots & \ddots & \ddots & \ddots & \ddots & \ddots & \vdots \\
    \vdots & \ddots & \ddots & \ddots & \ddots & \ddots & \ddots & \vdots \\
    0 & \cdots & \cdots & \cdots & 0 & \lambda & (1-2\lambda) & \lambda \\
    0 & \cdots & \cdots & \cdots & \cdots & 0 & \lambda & (1-2\lambda)
\end{bmatrix}$$

3. Siapkan matriks grid $w$ berukuran $(m+1) \times N$ untuk menyimpan $w_{i,j}$.

    Isi semua dengan nol terlebih dahulu, atau setidaknya, pastikan baris pertama dan baris terakhir terisi nol semua (sesuai syarat batas).

    Misalkan kolom ke-$j$, kecuali baris pertama dan baris terakhir, ditulis $\textbf{w}^{(j)}$, yaitu

    $$\textbf{w}^{(j)} = (w_{2,j}, w_{3,j}, \dots, w_{m,j})$$

4. Isi kolom pertama $w$ berdasarkan nilai awal, yaitu

    $$w_{i,1} = f(x_i)$$

    atau bisa ditulis

    $$\textbf{w}^{(0)} = (f(x_2), f(x_3), \dots, f(x_m))$$

5. Untuk $j = 2, \dots, N$, isi kolom ke-$j$ dengan perkalian matriks

    $$\textbf{w}^{(j)} = A\textbf{w}^{(j-1)}$$

### *Function file*

In [ ]:
function [x, t, u] = explicitheat(d, f, lb, rb, xb, xu, tb, tu, dx, dt)
  x = xb:dx:xu;
  t = tb:dt:tu;
  u = [];
  nt = length(t);
  nx = length(x);

  lambd = (d * dt) / dx^2;

  for j = 1:nx
    u(j, 1) = f(x(j));
  endfor

  for n = 2:nt
    u(1, n) = lb(t(n));
    u(nx, n) = rb(t(n));
  endfor

  for n = 1:nt-1
    for j = 2:nx-1
      u(j, n+1) = (1 - 2*lambd) * u(j, n) + lambd * (u(j+1, n) + u(j-1, n));
    endfor
  endfor
endfunction

### Contoh soal

Akan kita uji menggunakan persamaan panas:

\begin{align*}
    u_t - u_{xx} &= 0, \quad 0 < x < 1, \quad t > 0 \\
    u(0,t) &= u(1,t) = 0, \quad t \le 0 \\
    u(x,0) &= 10x^3(1-x), \quad 0 \le x \le 1 \\
\end{align*}

Solusi eksak dari PDP tersebut adalah:

\begin{align*}
    u(x,t) &= \sum_{n=1}^{\infty} c_n e^{-n^2 \pi^2 t} \sin \left( n\pi x\right) \\
    c_n &= 20 \int_0^1 x^3 (1-x) \sin \left( n\pi x \right) dx, \quad n = 1, 2, \dots
\end{align*}

Untuk keperluan komputasi, akan kita ambil 10 suku pertama dari ekspansi deret Fourier dari u(x,t).

In [ ]:
d = 1;
f = @(x) 10 * x^3 * (1-x);
lb = rb = @(t) 0;
xb = 0;
xu = 1;
tb = 0;
tu = 1;
dx = 0.2;
dt = 0.02;

[x, t, u] = explicitheat(d, f, lb, rb, xb, xu, tb, tu, dx, dt);

for i = 1:length(x)
  for j = 1:length(t)
    w(i, j) = 0;
    for n = 1:10
      F = @(x) x.^3 .* (1-x) .* sin(n.*pi.*x);
      cn(n) = 20 * integral(F, 0, 1);
      w(i, j) += cn(n) * exp(-n^2*pi^2*t(j)) * sin(n*pi*x(i));
    endfor
  endfor
endfor

figure(1);
mesh(x, t, u');
xlabel("x");
ylabel("t");
zlabel("u");

figure(1);
mesh(x, t, w');
xlabel("x");
ylabel("t");
zlabel("w");

figure(3);
for j = 1:length(t)
  plot(x, u(:, j), 'k', 'linewidth', 1.5);
  ylim([0, 1.5]);
  title("Animasi solusi aproksimasi u(x, t) seiring berjalannya t");
  drawnow;
  pause(0.1);
endfor

## Metode *Backward Difference* untuk PDP Parabolik

### Ide utama

### *Function file*

In [ ]:
function [x, t, w] = ImplicitHeat(d, f, lb, rb, xb, xu, tb, tu, dx, dt)
  x = xb:dx:xu;
  t = tb:dt:tu;
  nx = length(x);
  nt = length(t);

  % Nilai lambda
  lambd = (d * dt) / (dx^2);

  % Nilai awal dan syarat batas
  for i = 1:nx
    w(i, 1) = f(x(i));
  endfor

  for j = 2:nt
    w(1, j) = lb(t(j));
    w(nx, j) = rb(t(j));
  endfor

  % Penyelesaian SPL dengan faktorisasi Crout
  l(2) = 1 + 2*lambd;
  u(2) = -lambd / l(2);
  for i = 3:nx-2
    l(i) = 1 + 2*lambd + lambd*u(i-1);
    u(i) = -lambd / l(i);
  endfor
  l(nx-1) = 1 + 2*lambd + lambd*u(nx-2);
  for j = 2:nt
    z(2) = w(2, j-1) / l(2);
    for i = 3:nx-1
      z(i) = (w(i, j-1) + lambd*z(i-1)) / l(i);
    endfor
    w(nx-1, j) = z(nx-1);
    for i = nx-2:-1:2
      w(i, j) = z(i) - u(i)*w(i+1, j);
    endfor
  endfor
endfunction

### Contoh soal

Akan kita uji dengan persamaan difusi:

\begin{align*}
    u_t - u_{xx} &= 0, \quad 0 < x < 1, \quad t > 0, \\
    u(0,t) &= u(1,t) = 0, \quad t > 0, \\
    u(x,0) &= \sin \left(\pi x\right), \quad 0 \le x \le 1
\end{align*}

dengan solusi eksak:

$$u(x,t) = e^{-\pi^2 t} \sin \left(\pi x\right)$$

Kita batasi $t$ menjadi $0 \le t \le 1$ dan gunakan $\Delta x = 0.1$ dan $\Delta k = 0.01$, di mana kondisinya tidak stabil untuk metode eksplisit.

In [ ]:
d = 1;
f = @(x) sin(pi*x);
lb = rb = @(t) 0;
xb = 0;
xu = 1;
tb = 0;
tu = 0.5;
dx = 0.1;
dt = 0.01;

[x, t, w] = ImplicitHeat(d, f, lb, rb, xb, xu, tb, tu, dx, dt);

u = @(x, t) exp(-pi^2.*t) * sin(pi.*x);
for i = 1:length(x)
  for j = 1:length(t)
    ufig(i, j) = u(x(i), t(j));
  endfor
endfor

figure(1);
mesh(x, t, ufig');
xlabel("x");
ylabel("t");
zlabel("u");

figure(2);
mesh(x, t, w');
xlabel("x");
ylabel("t");
zlabel("u");

## Metode Crack-Nicolson untuk PDP Parabolik

### Ide utama

### *Function file*

In [ ]:
function [x, t, w] = CrankNicolson(d, f, lb, rb, xb, xu, tb, tu, dx, dt)
  x = xb:dx:xu;
  t = tb:dt:tu;
  nx = length(x);
  nt = length(t);

  % Nilai lambda
  lambd = (d * dt) / (dx^2);

  % Nilai awal dan syarat batas
  for i = 1:nx
    w(i, 1) = f(x(i));
  endfor

  for j = 2:nt
    w(1, j) = lb(t(j));
    w(nx, j) = rb(t(j));
  endfor

  % Penyelesaian SPL menggunakan faktorisasi Crout
  l(2) = 1 + lambd;
  u(2) = -lambd / (2*l(2));
  for i = 3:nx-2
    l(i) = 1 + lambd + (lambd*u(i-1))/2;
    u(i) = -lambd / (2*l(i));
  endfor
  l(nx-1) = 1 + lambd + (lambd*u(nx-2))/2;
  for j = 2:nt
    z(2) = ((1-lambd)*w(2, j-1) + (lambd/2)*w(3, j-1)) / l(2);
    for i = 3:nx-1
      z(i) = ((1-lambd)*w(i, j-1) + (lambd/2)*(w(i+1, j-1) + w(i-1, j-1) + z(i-
      1))) / l(i);
    endfor
    w(nx-1, j) = z(nx-1);
    for i = nx-2:-1:2
      w(i, j) = z(i) - u(i)*w(i+1, j);
    endfor
  endfor
endfunction

### Contoh soal

Sama seperti untuk *backward difference*, akan kita uji dengan persamaan difusi:

\begin{align*}
    u_t - u_{xx} &= 0, \quad 0 < x < 1, \quad t > 0, \\
    u(0,t) &= u(1,t) = 0, \quad t > 0, \\
    u(x,0) &= \sin \left(\pi x\right), \quad 0 \le x \le 1
\end{align*}

dengan solusi eksak:

$$u(x,t) = e^{-\pi^2 t} \sin \left(\pi x\right)$$

Kita batasi $t$ menjadi $0 \le t \le 1$ dan gunakan $\Delta x = 0.1$ dan $\Delta k = 0.01$, di mana kondisinya tidak stabil untuk metode eksplisit.

In [ ]:
d = 1;
f = @(x) sin(pi*x);
lb = rb = @(t) 0;
xb = 0;
xu = 1;
tb = 0;
tu = 0.5;
dx = 0.1;
dt = 0.01;

[x, t, w] = CrankNicolson(d, f, lb, rb, xb, xu, tb, tu, dx, dt);

u = @(x, t) exp(-pi^2.*t) * sin(pi.*x);
for i = 1:length(x)
  for j = 1:length(t)
    ufig(i, j) = u(x(i), t(j));
  endfor
endfor

figure(1);
mesh(x, t, ufig');
xlabel("x");
ylabel("t");
zlabel("u");

figure(2);
mesh(x, t, w');
xlabel("x");
ylabel("t");
zlabel("u");